In [ ]:
#!pip install tensorflow_hub

In [ ]:
import pandas as pd
import os
import numpy as np
import time

import PIL.Image as Image
import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

from keras.preprocessing.image import load_img

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from matplotlib.backends.backend_pdf import PdfPages

In [ ]:
path_input_images = r'C:\Users\61431\CV\Task 5.2 HD\ImageNet\Input'
path_input_noisy_images  = r'C:\Users\61431\CV\Task 5.2 HD\ImageNet\Noisy_Input'
path_artistic = r'C:\Users\61431\CV\Task 5.2 HD\ImageNet\artistic'

# Data Preparation - labeling


path1 = r'C:\Users\61431\CV\Task 5.2 HD\ImageNet\input_dir'
path2 = r'C:\Users\61431\CV\Task 5.2 HD\ImageNet\daisy'
path3 = r'C:\Users\61431\CV\Task 5.2 HD\ImageNet\dandelion'
path4 = r'C:\Users\61431\CV\Task 5.2 HD\ImageNet\rose'
path5 = r'C:\Users\61431\CV\Task 5.2 HD\ImageNet\sunflower'

df = pd.DataFrame(columns=['image','label'])

list_of_img = [os.listdir(path1),os.listdir(path2),os.listdir(path3),os.listdir(path4),os.listdir(path5)]


count = 1
for i in list_of_img:
    for j in i:
        if count == 1:
            df = df.append({'image':j,'label':''}, ignore_index = True)
        elif count == 2:
            df = df.append({'image':j,'label':'daisy'}, ignore_index = True)
        elif count == 3:
            df = df.append({'image':j,'label':'dandelion'}, ignore_index = True)
        elif count == 4:
            df = df.append({'image':j,'label':'rose'}, ignore_index = True)
        elif count == 5:
            df = df.append({'image':j,'label':'sunflower'}, ignore_index = True)
    count+= 1
    

in_labels ={}
with open('map_clsloc.txt', 'r') as t:
    for line in t:
        line = line.strip().split()
        curr_word = line[0]
        label = line[2]
        in_labels[curr_word] = label

for i in range(0, 100):
    s = df.iloc[i, 0]
    s = s[:-5]
    df.iloc[i, 1] = in_labels[s]
    
    
df.to_csv('labels.csv')

df1 = pd.DataFrame(columns=['image','label'])

for i in os.listdir(path_artistic):
    df1 = df1.append({'image':i,'label':i[:-4]}, ignore_index = True)
    
df1.to_csv('labels_artistic.csv')



## Image Recognition Using MobileNetV2 model (ImageNet Classifier)

In [ ]:
def load_image(img):
    input_img = load_img(img)
    input_img = np.array(input_img) 
    input_img = tf.image.resize(input_img, IMAGE_SHAPE)
    input_img = np.array(input_img)/255.0
    return input_img

def predict_class(input_img):
    labels_path = tf.keras.utils.get_file('ImageNetLabels.txt','https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
    imagenet_labels = np.array(open(labels_path).read().splitlines())
    
    result = classifier.predict(input_img[np.newaxis, ...])
    
    top_5_predicted_classes = pd.DataFrame(columns=['Class', 'Confidence'])
    
    predicted_class = np.argmax(result[0], axis=-1)
    top_1_predicted_class = imagenet_labels[predicted_class]
    top_1_prob = np.max(result[0], axis=-1)
    
    
    predicted_classes = result[0].argsort()[-5:][::-1]
    probs = np.sort(result[0])[::-1][:5]
    for i, c in enumerate(predicted_classes):
        top_5_predicted_classes = top_5_predicted_classes.append({'Class':imagenet_labels[c], 'Confidence':probs[i]}, ignore_index = True)
    

    
    return top_5_predicted_classes

In [ ]:
# Read image label csv
df = pd.read_csv('labels.csv')
df1 = pd.read_csv('labels_artistic.csv')
# Import pretrained model mobilenet_v2
classifier_model ="https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4"

IMAGE_SHAPE = (224, 224)

classifier = tf.keras.Sequential([hub.KerasLayer(classifier_model, input_shape=IMAGE_SHAPE+(3,))])

In [ ]:
# # Classification Task 1 - Realistic images without noise

list_of_images = list(df['image'])

pdf = PdfPages(r'C:\Users\61431\CV\Task 5.2 HD\ImageNet\clean_img.pdf')

# Prediction
y_true = [each_string.lower() for each_string in list(df['label'])]
for e, i in enumerate(y_true):
    if '-' in i:
        i = i.replace("-", "")
    if "'" in i:
        i = i.replace("'", "")
    if "_" in i:
        i = i.replace("_", "")
    if " " in i:
        i = i.replace(" ", "")
    y_true[e] = i
        
y_pred_top_1 = []
y_pred_top_5 = []
for i in list_of_images:
    img = path_input_images + '\%s'%i
    input_img = load_image(img)
    top_5_predicted_classes = predict_class(input_img)
    
    fig = plt.figure(figsize=(10, 4))
    
    plt.subplot(121)
    plt.imshow(input_img)
    plt.gca().axis('off')
    
    plt.subplot(122)
    plt.bar(top_5_predicted_classes['Class'], top_5_predicted_classes['Confidence'], color='royalblue')
    plt.title("Class and Confidence")
    plt.xlabel("Class")
    plt.ylabel("Confidence")
    plt.tight_layout()
    
    pdf.savefig(fig)
    
    plt.show()
    plt.clf()
    
    y_pred_top_1.append(str(top_5_predicted_classes['Class'].iloc[0]).lower())
    y_pred_top_5.append([each_string.lower() for each_string in list(top_5_predicted_classes['Class'])])
    
    
    
pdf.close()


for e, i in enumerate(y_pred_top_1):
    if '-' in i:
        i = i.replace("-", "")
    if "'" in i:
        i = i.replace("'", "")
    if "_" in i:
        i = i.replace("_", "")
    if " " in i:
        i = i.replace(" ", "")
    y_pred_top_1[e] = i
    
for e, j in enumerate(y_pred_top_5):
    for d, i in enumerate(j):
        if '-' in i:
            i = i.replace("-", "")
        if "'" in i:
            i = i.replace("'", "")
        if "_" in i:
            i = i.replace("_", "")
        if " " in i:
            i = i.replace(" ", "")
        y_pred_top_5[e][d] = i


In [ ]:
# Top 1 error rate
count = 0
for i in range(len(y_true)):
    if y_true[i] == y_pred_top_1[i]:
        count += 1

print('Out of %i test images, %i were classicfied correctly'%(len(y_true), count))
top_1_error_rate = (1 - (count / len(y_true))) * 100
print('\n\nTop 1 error rate of ImageNet classifier MobileNetV2 model is %f%%'%top_1_error_rate)

# Top 5 error rate
count = 0
for i in range(len(y_true)):
    if y_true[i] in y_pred_top_5[i]:
        count += 1
top_1_error_rate = (1 - (count / len(y_true)))* 100
print('\n\nTop 5 error rate of ImageNet classifier MobileNetV2 model is %f%%'%top_1_error_rate)
        

In [ ]:
# Classification Task 4 - Artistic images without noise

list_of_images_artistic = list(df1['image'])

pdf_2 = PdfPages(r'C:\Users\61431\CV\Task 5.2 HD\ImageNet\artistic_img.pdf')

# Prediction
y_true = [each_string.lower() for each_string in list(df1['label'])]
for e, i in enumerate(y_true):
    if '-' in i:
        i = i.replace("-", "")
    if "'" in i:
        i = i.replace("'", "")
    if "_" in i:
        i = i.replace("_", "")
    if " " in i:
        i = i.replace(" ", "")
    y_true[e] = i
        
y_pred_top_1 = []
y_pred_top_5 = []

for i in list_of_images_artistic:
    img = path_artistic + '\%s'%i
    input_img = load_image(img)
    top_5_predicted_classes = predict_class(input_img)
    
    fig = plt.figure(figsize=(10, 4))
    
    plt.subplot(121)
    plt.imshow(input_img)
    plt.gca().axis('off')
    
    plt.subplot(122)
    plt.bar(top_5_predicted_classes['Class'], top_5_predicted_classes['Confidence'], color='royalblue')
    plt.title("Class and Confidence")
    plt.xlabel("Class")
    plt.ylabel("Confidence")
    plt.tight_layout()
    
    pdf_2.savefig(fig)
    
    plt.show()
    plt.clf()
    
    y_pred_top_1.append(str(top_5_predicted_classes['Class'].iloc[0]).lower())
    y_pred_top_5.append([each_string.lower() for each_string in list(top_5_predicted_classes['Class'])])
    
    
    
pdf_2.close()


for e, i in enumerate(y_pred_top_1):
    if '-' in i:
        i = i.replace("-", "")
    if "'" in i:
        i = i.replace("'", "")
    if "_" in i:
        i = i.replace("_", "")
    if " " in i:
        i = i.replace(" ", "")
    y_pred_top_1[e] = i
    
for e, j in enumerate(y_pred_top_5):
    for d, i in enumerate(j):
        if '-' in i:
            i = i.replace("-", "")
        if "'" in i:
            i = i.replace("'", "")
        if "_" in i:
            i = i.replace("_", "")
        if " " in i:
            i = i.replace(" ", "")
        y_pred_top_5[e][d] = i



In [ ]:
# Top 1 error rate
count = 0
for i in range(len(y_true)):
    if y_true[i] == y_pred_top_1[i]:
        count += 1

print('Out of %i test images, %i were classicfied correctly'%(len(y_true), count))
top_1_error_rate = (1 - (count / len(y_true))) * 100
print('\n\nTop 1 error rate of ImageNet classifier MobileNetV2 model is %f%%'%top_1_error_rate)

# Top 5 error rate
count = 0
for i in range(len(y_true)):
    if y_true[i] in y_pred_top_5[i]:
        count += 1
top_1_error_rate = (1 - (count / len(y_true)))* 100
print('\n\nTop 5 error rate of ImageNet classifier MobileNetV2 model is %f%%'%top_1_error_rate)
        

## Image Recognition Using MobileNetV2 model (ImageNet Classifier) for Noise Induced Images

In [ ]:
pretrained_model = tf.keras.applications.MobileNetV2(include_top=True,
                                                     weights='imagenet')
pretrained_model.trainable = False


In [ ]:
# Create adversarial pattern using fast gradient sign method (fgsm)
loss_object = tf.keras.losses.CategoricalCrossentropy()

def create_adversarial_pattern(input_image, input_label):
    with tf.GradientTape() as tape:
        tape.watch(input_image)
        prediction = pretrained_model(input_image)
        loss = loss_object(input_label, prediction)

    # Get the gradients of the loss w.r.t to the input image.
    gradient = tape.gradient(loss, input_image)
    
    # Get the sign of the gradients to create the perturbation
    signed_grad = tf.sign(gradient)
    return signed_grad 

def preprocess(image):
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, (224, 224))
    image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
    image = image[None, ...]
    return image

def load_image_1(img):
    input_img = load_img(img)
    input_img = np.array(input_img)
    return input_img

decode_predictions = tf.keras.applications.mobilenet_v2.decode_predictions

In [ ]:
image_path = tf.keras.utils.get_file('YellowLabradorLooking_new.jpg', 'https://storage.googleapis.com/download.tensorflow.org/example_images/YellowLabradorLooking_new.jpg')
image_raw = tf.io.read_file(image_path)
image = tf.image.decode_image(image_raw)

image = preprocess(image)
image_probs = pretrained_model.predict(image)

labrador_retriever_index = 208
label = tf.one_hot(labrador_retriever_index, image_probs.shape[-1])
label = tf.reshape(label, (1, image_probs.shape[-1]))

perturbations = create_adversarial_pattern(image, label)
plt.imshow(perturbations[0]*0.5+0.5);

In [ ]:
# Classification Task 2

esp = 0.1

y_pred_top_1 = []
y_pred_top_5 = []

list_of_images = list(df['image'])
pdf_1 = PdfPages(r'C:\Users\61431\CV\Task 5.2 HD\ImageNet\noisy_img_esp1.pdf')

y_true = [each_string.lower() for each_string in list(df['label'])]
for e, i in enumerate(y_true):
    if '-' in i:
        i = i.replace("-", "")
    if "'" in i:
        i = i.replace("'", "")
    if "_" in i:
        i = i.replace("_", "")
    if " " in i:
        i = i.replace(" ", "")
    y_true[e] = i
        

for i in list_of_images:
    img = path_input_images + '\%s'%i
    input_img = load_image_1(img)
    input_img = preprocess(input_img)
    adv_x = input_img + tf.math.scalar_mul(esp, perturbations)
    prediction = pretrained_model.predict(adv_x)
  
    predicted_classes = decode_predictions(prediction, top=5)[0]
    
    classp = []
    conf = []
    for i in range(len(predicted_classes)):
        classp.append(predicted_classes[i][1])
        conf.append(predicted_classes[i][2])
        
    
    fig = plt.figure(figsize=(10, 4))
    
    plt.subplot(121)
    plt.imshow(adv_x[0]*0.5+0.5)
    plt.gca().axis('off')
    
    plt.subplot(122)
    plt.bar(classp, conf, color='royalblue')
    plt.title("Class and Confidence")
    plt.xlabel("Class")
    plt.ylabel("Confidence")
    plt.tight_layout()
    
    pdf_1.savefig(fig)
    
    plt.show()
    plt.clf()
    
    y_pred_top_1.append(classp[0].lower())
    y_pred_top_5.append([each_string.lower() for each_string in classp])
    
    
pdf_1.close()
    
for e, i in enumerate(y_pred_top_1):
    if '-' in i:
        i = i.replace("-", "")
    if "'" in i:
        i = i.replace("'", "")
    if "_" in i:
        i = i.replace("_", "")
    if " " in i:
        i = i.replace(" ", "")
    y_pred_top_1[e] = i
    
for e, j in enumerate(y_pred_top_5):
    for d, i in enumerate(j):
        if '-' in i:
            i = i.replace("-", "")
        if "'" in i:
            i = i.replace("'", "")
        if "_" in i:
            i = i.replace("_", "")
        if " " in i:
            i = i.replace(" ", "")
        y_pred_top_5[e][d] = i

In [ ]:
# Top 1 error rate
count = 0
for i in range(len(y_true)):
    if y_true[i] == y_pred_top_1[i]:
        count += 1

print('Out of %i test images, %i were classicfied correctly'%(len(y_true), count))
top_1_error_rate = (1 - (count / len(y_true))) * 100
print('\n\nTop 1 error rate of ImageNet classifier MobileNetV2 model is %f%%'%top_1_error_rate)

# Top 5 error rate
count = 0
for i in range(len(y_true)):
    if y_true[i] in y_pred_top_5[i]:
        count += 1
top_1_error_rate = (1 - (count / len(y_true)))* 100
print('\n\nTop 5 error rate of ImageNet classifier MobileNetV2 model is %f%%'%top_1_error_rate)
        

In [ ]:
# Classification Task 3
esp = 0.2

y_pred_top_1 = []
y_pred_top_5 = []

list_of_images = list(df['image'])
pdf_1 = PdfPages(r'C:\Users\61431\CV\Task 5.2 HD\ImageNet\noisy_img_esp2.pdf')

y_true = [each_string.lower() for each_string in list(df['label'])]
for e, i in enumerate(y_true):
    if '-' in i:
        i = i.replace("-", "")
    if "'" in i:
        i = i.replace("'", "")
    if "_" in i:
        i = i.replace("_", "")
    if " " in i:
        i = i.replace(" ", "")
    y_true[e] = i
        

for i in list_of_images:
    img = path_input_images + '\%s'%i
    input_img = load_image_1(img)
    input_img = preprocess(input_img)
    adv_x = input_img + tf.math.scalar_mul(esp, perturbations)
    prediction = pretrained_model.predict(adv_x)
  
    predicted_classes = decode_predictions(prediction, top=5)[0]
    
    classp = []
    conf = []
    for i in range(len(predicted_classes)):
        classp.append(predicted_classes[i][1])
        conf.append(predicted_classes[i][2])
        
    
    fig = plt.figure(figsize=(10, 4))
    
    plt.subplot(121)
    plt.imshow(adv_x[0]*0.5+0.5)
    plt.gca().axis('off')
    
    plt.subplot(122)
    plt.bar(classp, conf, color='royalblue')
    plt.title("Class and Confidence")
    plt.xlabel("Class")
    plt.ylabel("Confidence")
    plt.tight_layout()
    
    pdf_1.savefig(fig)
    
    plt.show()
    plt.clf()
    
    y_pred_top_1.append(classp[0].lower())
    y_pred_top_5.append([each_string.lower() for each_string in classp])
    
    
pdf_1.close()
    
for e, i in enumerate(y_pred_top_1):
    if '-' in i:
        i = i.replace("-", "")
    if "'" in i:
        i = i.replace("'", "")
    if "_" in i:
        i = i.replace("_", "")
    if " " in i:
        i = i.replace(" ", "")
    y_pred_top_1[e] = i
    
for e, j in enumerate(y_pred_top_5):
    for d, i in enumerate(j):
        if '-' in i:
            i = i.replace("-", "")
        if "'" in i:
            i = i.replace("'", "")
        if "_" in i:
            i = i.replace("_", "")
        if " " in i:
            i = i.replace(" ", "")
        y_pred_top_5[e][d] = i

In [ ]:
# Top 1 error rate
count = 0
for i in range(len(y_true)):
    if y_true[i] == y_pred_top_1[i]:
        count += 1

print('Out of %i test images, %i were classicfied correctly'%(len(y_true), count))
top_1_error_rate = (1 - (count / len(y_true))) * 100
print('\n\nTop 1 error rate of ImageNet classifier MobileNetV2 model is %f%%'%top_1_error_rate)

# Top 5 error rate
count = 0
for i in range(len(y_true)):
    if y_true[i] in y_pred_top_5[i]:
        count += 1
top_1_error_rate = (1 - (count / len(y_true)))* 100
print('\n\nTop 5 error rate of ImageNet classifier MobileNetV2 model is %f%%'%top_1_error_rate)
        